# Basic preprocessing

# Table of Contents
- Load data
- Change column name
- Calculate the timestamp being forecasted in `fcst`
- Change column name of obs_data
- Preprocess wind (windspeed and winddirection)
- Final overview & save in pickle

## Load data

In [4]:
# load data
import pandas as pd
import numpy as np
from glob import glob
import pickle

path = '../original_dataset/'

files = sorted(glob(path+'*.csv'))

files

['../original_dataset/dangjin_fcst_data.csv',
 '../original_dataset/dangjin_obs_data.csv',
 '../original_dataset/energy.csv',
 '../original_dataset/sample_submission.csv',
 '../original_dataset/site_info.csv',
 '../original_dataset/ulsan_fcst_data.csv',
 '../original_dataset/ulsan_obs_data.csv']

In [5]:
dangjin_fcst_data = pd.read_csv(files[0])
dangjin_obs_data = pd.read_csv(files[1])
ulsan_fcst_data = pd.read_csv(files[5])
ulsan_obs_data = pd.read_csv(files[6])

## Change column name
1. obs_data의 '지점'과 '지점명'은 drop
2. Column name을 fcst_data를 기준으로 맞춘다.

In [6]:
# drop & re-order
dangjin_obs_data = dangjin_obs_data.loc[:,['일시','기온(°C)','습도(%)','풍속(m/s)','풍향(16방위)','전운량(10분위)']]
ulsan_obs_data = ulsan_obs_data.loc[:,['일시','기온(°C)','습도(%)','풍속(m/s)','풍향(16방위)','전운량(10분위)']]

In [7]:
# change column names
new_column_names = list(dangjin_fcst_data.columns)
new_column_names.remove('forecast')
new_column_names

dangjin_obs_data.columns = new_column_names
ulsan_obs_data.columns = new_column_names
dangjin_obs_data

,Forecast time,Temperature,Humidity,WindSpeed,WindDirection,Cloud
0,2018-03-01 00:00,3.1,96.0,3.6,340.0,NaN
1,2018-03-01 01:00,2.8,97.0,0.7,140.0,NaN
2,2018-03-01 02:00,2.6,95.0,3.2,320.0,NaN
3,2018-03-01 03:00,2.0,97.0,1.9,230.0,NaN
4,2018-03-01 04:00,2.2,97.0,2.1,180.0,NaN
...,...,...,...,...,...,...
25621,2021-01-31 19:00,6.7,75.0,1.5,200.0,8.0
25622,2021-01-31 20:00,6.2,77.0,0.8,200.0,8.0
25623,2021-01-31 21:00,5.3,82.0,0.7,230.0,8.0
25624,2021-01-31 22:00,5.8,77.0,1.0,200.0,8.0


## Calculate the timestamp being forecasted in `fcst`
: fcst_data에서 Forecast_time과 forecast를 더해 예측 대상이 되는 timestamp를 새로운 column으로  

1. Convert dtype to pd.datetime
2. Map a timestampe-adding function to the timestamp column

**'Forecast time' column은 남겨준다. 이후에 merge_to_recent를 위함.**

In [8]:
# convert dtype to pd.datetime 
dangjin_fcst_data['Forecast time'] = pd.to_datetime(dangjin_fcst_data['Forecast time'])
dangjin_obs_data['Forecast time'] = pd.to_datetime(dangjin_obs_data['Forecast time'])
ulsan_fcst_data['Forecast time'] = pd.to_datetime(ulsan_fcst_data['Forecast time'])
ulsan_obs_data['Forecast time'] = pd.to_datetime(ulsan_obs_data['Forecast time'])

In [9]:
# calculate
def to_date(x):
    return pd.DateOffset(hours=x)

dangjin_fcst_data['time'] = dangjin_fcst_data['Forecast time'] + dangjin_fcst_data['forecast'].map(to_date)
ulsan_fcst_data['time'] = ulsan_fcst_data['Forecast time'] + ulsan_fcst_data['forecast'].map(to_date)

/home/yejoon/anaconda3/lib/python3.8/site-packages/pandas/core/arrays/datetimelike.py:1108: PerformanceWarning: Adding/subtracting object-dtype array to DatetimeArray not vectorized
  warnings.warn(


In [10]:
# drop the 'forecast' column
dangjin_fcst_data = dangjin_fcst_data.drop('forecast', axis=1)
ulsan_fcst_data = ulsan_fcst_data.drop('forecast', axis=1)

In [11]:
dangjin_fcst_data

,Forecast time,Temperature,Humidity,WindSpeed,WindDirection,Cloud,time
0,2018-03-01 11:00:00,0.0,60.0,7.3,309.0,2.0,2018-03-01 15:00:00
1,2018-03-01 11:00:00,-2.0,60.0,7.1,314.0,1.0,2018-03-01 18:00:00
2,2018-03-01 11:00:00,-2.0,60.0,6.7,323.0,1.0,2018-03-01 21:00:00
3,2018-03-01 11:00:00,-2.0,55.0,6.7,336.0,1.0,2018-03-02 00:00:00
4,2018-03-01 11:00:00,-4.0,55.0,5.5,339.0,1.0,2018-03-02 03:00:00
...,...,...,...,...,...,...,...
162203,2021-03-01 08:00:00,7.0,40.0,3.2,187.0,1.0,2021-03-03 12:00:00
162204,2021-03-01 08:00:00,8.0,40.0,4.5,217.0,1.0,2021-03-03 15:00:00
162205,2021-03-01 08:00:00,5.0,55.0,2.2,210.0,1.0,2021-03-03 18:00:00
162206,2021-03-01 08:00:00,1.0,80.0,1.9,164.0,1.0,2021-03-03 21:00:00


In [12]:
# re-order
new_col_order = ['time', 'Forecast time', 'Temperature', 'Humidity', 'WindSpeed', 'WindDirection', 'Cloud']

dangjin_fcst_data = dangjin_fcst_data.loc[:,new_col_order]
ulsan_fcst_data = ulsan_fcst_data.loc[:,new_col_order]

In [13]:
dangjin_fcst_data

,time,Forecast time,Temperature,Humidity,WindSpeed,WindDirection,Cloud
0,2018-03-01 15:00:00,2018-03-01 11:00:00,0.0,60.0,7.3,309.0,2.0
1,2018-03-01 18:00:00,2018-03-01 11:00:00,-2.0,60.0,7.1,314.0,1.0
2,2018-03-01 21:00:00,2018-03-01 11:00:00,-2.0,60.0,6.7,323.0,1.0
3,2018-03-02 00:00:00,2018-03-01 11:00:00,-2.0,55.0,6.7,336.0,1.0
4,2018-03-02 03:00:00,2018-03-01 11:00:00,-4.0,55.0,5.5,339.0,1.0
...,...,...,...,...,...,...,...
162203,2021-03-03 12:00:00,2021-03-01 08:00:00,7.0,40.0,3.2,187.0,1.0
162204,2021-03-03 15:00:00,2021-03-01 08:00:00,8.0,40.0,4.5,217.0,1.0
162205,2021-03-03 18:00:00,2021-03-01 08:00:00,5.0,55.0,2.2,210.0,1.0
162206,2021-03-03 21:00:00,2021-03-01 08:00:00,1.0,80.0,1.9,164.0,1.0


## Change column name of obs_data
: from 'Forecast time' to 'time'  
This is convenient becuase 'time' is the column name in 'energy.csv'.  

In [14]:
dangjin_obs_data = dangjin_obs_data.rename(columns={'Forecast time':'time'})
ulsan_obs_data = ulsan_obs_data.rename(columns={'Forecast time':'time'})

## Preprocess wind(wind_speed and wind_direction) data

In [15]:
def preprocess_wind(data):
    '''
    data: pd.DataFrmae which contains the columns 'WindSpeed' and 'WindDirection'
    '''

    # degree to radian
    wind_direction_radian = data['WindDirection'] * np.pi / 180

    # polar coordinate to cartesian coordinate
    wind_x = data['WindSpeed'] * np.cos(wind_direction_radian)
    wind_y = data['WindDirection'] * np.sin(wind_direction_radian)

    # name pd.series
    wind_x.name = 'Wind_X'
    wind_y.name = 'Wind_Y'

    return wind_x, wind_y

In [16]:
# join 'Wind_X' and 'Wind_Y'
dangjin_fcst_data = dangjin_fcst_data.join(preprocess_wind(dangjin_fcst_data))
dangjin_obs_data = dangjin_obs_data.join(preprocess_wind(dangjin_obs_data))
ulsan_fcst_data = ulsan_fcst_data.join(preprocess_wind(ulsan_fcst_data))
ulsan_obs_data = ulsan_obs_data.join(preprocess_wind(ulsan_obs_data))

# drop 'WindSpeed' and 'WindDirection'
dangjin_fcst_data.drop(columns = ['WindSpeed','WindDirection'], inplace=True)
dangjin_obs_data.drop(columns = ['WindSpeed','WindDirection'], inplace=True)
ulsan_fcst_data.drop(columns = ['WindSpeed','WindDirection'], inplace=True)
ulsan_obs_data.drop(columns = ['WindSpeed','WindDirection'], inplace=True)


## Final overview & save in pickle

In [19]:
dangjin_fcst_data

,time,Forecast time,Temperature,Humidity,Cloud,Wind_X,Wind_Y
0,2018-03-01 15:00:00,2018-03-01 11:00:00,0.0,60.0,2.0,4.594039,-240.138102
1,2018-03-01 18:00:00,2018-03-01 11:00:00,-2.0,60.0,1.0,4.932074,-225.872697
2,2018-03-01 21:00:00,2018-03-01 11:00:00,-2.0,60.0,1.0,5.350858,-194.386252
3,2018-03-02 00:00:00,2018-03-01 11:00:00,-2.0,55.0,1.0,6.120755,-136.663512
4,2018-03-02 03:00:00,2018-03-01 11:00:00,-4.0,55.0,1.0,5.134692,-121.486735
...,...,...,...,...,...,...,...
162203,2021-03-03 12:00:00,2021-03-01 08:00:00,7.0,40.0,1.0,-3.176148,-22.789567
162204,2021-03-03 15:00:00,2021-03-01 08:00:00,8.0,40.0,1.0,-3.593860,-130.593860
162205,2021-03-03 18:00:00,2021-03-01 08:00:00,5.0,55.0,1.0,-1.905256,-105.000000
162206,2021-03-03 21:00:00,2021-03-01 08:00:00,1.0,80.0,1.0,-1.826397,45.204526


In [18]:
dangjin_obs_data

,time,Temperature,Humidity,Cloud,Wind_X,Wind_Y
0,2018-03-01 00:00:00,3.1,96.0,NaN,3.382893,-1.162868e+02
1,2018-03-01 01:00:00,2.8,97.0,NaN,-0.536231,8.999027e+01
2,2018-03-01 02:00:00,2.6,95.0,NaN,2.451342,-2.056920e+02
3,2018-03-01 03:00:00,2.0,97.0,NaN,-1.221296,-1.761902e+02
4,2018-03-01 04:00:00,2.2,97.0,NaN,-2.100000,2.204364e-14
...,...,...,...,...,...,...
25621,2021-01-31 19:00:00,6.7,75.0,8.0,-1.409539,-6.840403e+01
25622,2021-01-31 20:00:00,6.2,77.0,8.0,-0.751754,-6.840403e+01
25623,2021-01-31 21:00:00,5.3,82.0,8.0,-0.449951,-1.761902e+02
25624,2021-01-31 22:00:00,5.8,77.0,8.0,-0.939693,-6.840403e+01


In [116]:
ulsan_fcst_data

,time,Forecast time,Temperature,Humidity,Cloud,Wind_X,Wind_Y
0,2018-03-01 15:00:00,2018-03-01 11:00:00,8.0,20.0,2.0,6.572602,-263.118383
1,2018-03-01 18:00:00,2018-03-01 11:00:00,4.0,20.0,2.0,2.018728,-263.118383
2,2018-03-01 21:00:00,2018-03-01 11:00:00,3.0,30.0,2.0,1.195709,-240.138102
3,2018-03-02 00:00:00,2018-03-01 11:00:00,0.0,40.0,2.0,1.114717,-212.783533
4,2018-03-02 03:00:00,2018-03-01 11:00:00,-1.0,45.0,2.0,1.108191,-242.707312
...,...,...,...,...,...,...,...
162203,2021-03-03 12:00:00,2021-03-01 08:00:00,9.0,45.0,3.0,0.377795,82.381331
162204,2021-03-03 15:00:00,2021-03-01 08:00:00,9.0,45.0,3.0,-1.075104,103.627427
162205,2021-03-03 18:00:00,2021-03-01 08:00:00,8.0,55.0,3.0,-1.165822,103.461868
162206,2021-03-03 21:00:00,2021-03-01 08:00:00,6.0,65.0,3.0,-0.590453,98.866955


In [117]:
ulsan_obs_data

,time,Temperature,Humidity,Cloud,Wind_X,Wind_Y
0,2018-03-01 00:00:00,8.2,98.0,10.0,3.664801,-116.286849
1,2018-03-01 01:00:00,7.0,97.0,10.0,3.140782,-205.692035
2,2018-03-01 02:00:00,6.5,80.0,NaN,2.017919,-272.510860
3,2018-03-01 03:00:00,6.2,79.0,3.0,3.523804,-205.692035
4,2018-03-01 04:00:00,6.7,73.0,1.0,3.447200,-205.692035
...,...,...,...,...,...,...
25627,2021-01-31 19:00:00,8.8,50.0,5.0,-2.349232,-68.404029
25628,2021-01-31 20:00:00,8.7,49.0,1.0,-3.664801,-68.404029
25629,2021-01-31 21:00:00,8.4,51.0,7.0,-1.542690,-176.190222
25630,2021-01-31 22:00:00,9.4,51.0,8.0,-2.121199,-176.190222


In [118]:
# pickle the data
with open('pickles/dangjin_fcst_data.pkl','wb') as f:
    pickle.dump(dangjin_fcst_data,f)
with open('pickles/dangjin_obs_data.pkl','wb') as f:
    pickle.dump(dangjin_obs_data,f)
with open('pickles/ulsan_fcst_data.pkl','wb') as f:
    pickle.dump(ulsan_fcst_data,f)
with open('pickles/ulsan_obs_data.pkl','wb') as f:
    pickle.dump(ulsan_obs_data,f)